# Create training movies from csv file
In this notebook, we create training movies using the csv files generated by our MouseClick Fiji plugin which writes the event location as t,x,y,boolean format. The user has to input the image path used for creating the csv file along with the csv file. Other parameters to be specified by the user are indicated preceeding their respective blocks.

In [1]:
import numpy as np
from tifffile import imread 
import sys
import os
import cv2
from glob import glob
from tqdm import tqdm
sys.path.append("../NEAT")
from  NEATUtils import Augmentation
from NEATUtils import MovieCreator, npzfileGenerator, Upsample
try:
    from pathlib import Path
    Path().expanduser()
except (ImportError,AttributeError):
    from pathlib2 import Path

try:
    import tempfile
    tempfile.TemporaryDirectory
except (ImportError,AttributeError):
    from backports import tempfile


In the block below specify the image used fore creating the csv files along with the total categories for classification. 

The default categories are: Norman events , Apoptosis, Division, Macrocheate, NonMature and Mature P1 cells. 

Crop size specifies the XY crop to be created at the location of the dot in the csv file and the third dimension is the crop in time relative to the click location, so it is X,Y,T, Crops image area sizeX = -X/2:X/2, -Y/2:Y/2, -T:T, so 7 Frame movies if T = 3

Since the network learns the joint task of classification and localization, we need to create that anwser int eh training data itself. Hence user specifies the value of shift in pixels to shift the event to, the program moves the event location from the center to nine locations (left, right, bottom, top, upper-left/right and bottom-left/right)

The task of the program below is to create movie crops and assigned training label to each crop, labels used for events are:

Normal Event: Label 0
Apoptosis Event: Label 1
Division Event: Label 2
Macrocheate Event: Label 3
Non MatureP1 Event : Label 4
Mature Event: Label 5

So for division in the center the training label would be
[0,0,1,0,0,0.5,0.5,0.5]
for the same event on the top Left
[0,0,1,0,0,0.38,0.38,0.5] and so on. This task is done by the program automatically

In [2]:
###### Specify the image used for making the csv file
SourceDir = '/home/sancere/VarunNewton/TrainingDataONEATBin2/CSVforNeat/NEATcsvfiles/'
SourceImage = imread(SourceDir + 'EventMovie.tif')
NormalCSV = SourceDir + 'Movie2Normal.csv'
ApoptosisCSV = SourceDir +  'Movie2Apoptosis.csv'
DivisionCSV = SourceDir +  'Movie2Division.csv'
MacrocheateCSV = SourceDir +  'Movie2MacroKitty.csv'
NonMatureCSV = SourceDir +  'Movie2NonMature.csv'
MatureCSV = SourceDir +  'Movie2Mature.csv'

Rawsave_dir = '/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/CenterONEAT/'
Normalsave_dir = Rawsave_dir + 'NormalEvents'
Apoptosissave_dir = Rawsave_dir + 'ApoptosisEvents'
Divisionsave_dir = Rawsave_dir + 'DivisionEvents'
Macrocheatesave_dir = Rawsave_dir + 'MacroKittyEvents'
NonMaturesave_dir = Rawsave_dir + 'MatureEvents'
Maturesave_dir = Rawsave_dir + 'NonMatureEvents'

Path(Rawsave_dir).mkdir(exist_ok = True)
Path(Normalsave_dir).mkdir(exist_ok = True)
Path(Apoptosissave_dir).mkdir(exist_ok = True)
Path(Divisionsave_dir).mkdir(exist_ok = True)
Path(Macrocheatesave_dir).mkdir(exist_ok = True)
Path(NonMaturesave_dir).mkdir(exist_ok = True)
Path(Maturesave_dir).mkdir(exist_ok = True)

SaveNpzDirectory = '/home/sancere/VarunNewton/CurieTrainingDatasets/O-NEAT/'

crop_size = [54,54,5,3]
SizeX = crop_size[0]
SizeY = crop_size[1]
#Shift the event by these many pixels
shift = 0
TotalCategories = 6
Downsample = 1

In [ ]:
if Downsample == 2:
        scale_percent = 50 # percent of original size
        width = int(SourceImage.shape[2] * scale_percent / 100)
        height = int(SourceImage.shape[1] * scale_percent / 100)
        dim = (width, height)
        # resize image
        smallSourceImage = np.zeros([SourceImage.shape[0],height,width], dtype = 'float32')
        for i in tqdm(range(0, SourceImage.shape[0])):
          smallSourceImage[i,:] = cv2.resize(SourceImage[i,:], dim)
        SourceImage = smallSourceImage  


#For Normal Events/Negative controls
MovieCreator.CreateMoviesTXYZ(NormalCSV, SourceImage, crop_size, shift, TotalCategories, 0 ,Normalsave_dir, 'ONEATBIG')
#For Apoptosis
MovieCreator.CreateMoviesTXYZ(ApoptosisCSV, SourceImage, crop_size, shift, TotalCategories, 1 ,Apoptosissave_dir, 'ONEATBIG')
#For Division
MovieCreator.CreateMoviesTXYZ(DivisionCSV, SourceImage, crop_size, shift, TotalCategories, 2 ,Divisionsave_dir, 'ONEATBIG')
#For Macrocheate
MovieCreator.CreateMoviesTXYZ(MacrocheateCSV, SourceImage, crop_size, shift, TotalCategories, 3 ,Macrocheatesave_dir, 'ONEATBIG')
#For NonMatureP1
MovieCreator.CreateMoviesTXYZ(NonMatureCSV, SourceImage, crop_size, shift, TotalCategories, 4 ,NonMaturesave_dir, 'ONEATBIG')
#For MatureP1
MovieCreator.CreateMoviesTXYZ(MatureCSV, SourceImage, crop_size, shift, TotalCategories, 5 ,Maturesave_dir, 'ONEATBIG')



In [ ]:
SourceDir = '/home/sancere/VarunNewton/TrainingDataONEATBin2/CSVforNeat/NEATcsvfiles/'
SourceImage = imread(SourceDir + 'wt_N10.tif')
ApoptosisCSV = SourceDir +  'wt_N10Apoptosis.csv'
DivisionCSV = SourceDir +  'wt_N10Division.csv'
Downsample = 2


In [ ]:
if Downsample == 2:
        scale_percent = 50 # percent of original size
        width = int(SourceImage.shape[2] * scale_percent / 100)
        height = int(SourceImage.shape[1] * scale_percent / 100)
        dim = (width, height)
        # resize image
        smallSourceImage = np.zeros([SourceImage.shape[0],height,width], dtype = 'float32')
        for i in tqdm(range(0, SourceImage.shape[0])):
          smallSourceImage[i,:] = cv2.resize(SourceImage[i,:], dim)
        SourceImage = smallSourceImage  


#For Apoptosis
MovieCreator.CreateMoviesTXYZ(ApoptosisCSV, SourceImage, crop_size, shift, TotalCategories, 1 ,Apoptosissave_dir, 'ONEATWT')
#For Division
MovieCreator.CreateMoviesTXYZ(DivisionCSV, SourceImage, crop_size, shift, TotalCategories, 2 ,Divisionsave_dir, 'ONEATWT')


In [3]:
#elasticDeform = True, putNoise (Blur) = True in that order
Subdir = next(os.walk(Normalsave_dir))

for x in Subdir[1]:
    currentdir = Normalsave_dir + '/' + x
    Augmentation(currentdir,Normalsave_dir +'/' + x, SizeX, SizeY, False,True,AppendName = 'Bin2')
    
Subdir = next(os.walk(Apoptosissave_dir))

for x in Subdir[1]:
    
    currentdir = Apoptosissave_dir + '/' + x
    Augmentation(currentdir,Apoptosissave_dir +'/' + x, SizeX, SizeY, False,True,AppendName = 'Bin2') 
    
Subdir = next(os.walk(Divisionsave_dir))

for x in Subdir[1]:
    
    currentdir = Divisionsave_dir + '/' + x
    Augmentation(currentdir,Divisionsave_dir +'/' + x, SizeX, SizeY, False,False,AppendName = 'Bin2') 
    
Subdir = next(os.walk(Macrocheatesave_dir))

for x in Subdir[1]:
    
    currentdir = Macrocheatesave_dir + '/' + x
    Augmentation(currentdir,Macrocheatesave_dir +'/' + x, SizeX, SizeY, False,True,AppendName = 'Bin2') 

Subdir = next(os.walk(NonMaturesave_dir))

for x in Subdir[1]:
    
    currentdir = NonMaturesave_dir + '/' + x
    Augmentation(currentdir,NonMaturesave_dir +'/' + x, SizeX, SizeY, False,True,AppendName = 'Bin2') 
    
Subdir = next(os.walk(Maturesave_dir))

for x in Subdir[1]:
    
    currentdir = Maturesave_dir + '/' + x
    Augmentation(currentdir,Maturesave_dir +'/' + x, SizeX, SizeY, False,True,AppendName = 'Bin2') 
        
    

In [4]:
DirectoryList = []
LabelList = []

Subdir = next(os.walk(Normalsave_dir))

for x in Subdir[1]:
    currentdir = Normalsave_dir + '/' + x
    CsvFile = sorted(glob(currentdir + '/' + '*.csv'))
    Labels = np.loadtxt(CsvFile[0], unpack = True)
    SubSubdir = next(os.walk(currentdir))
    for y in SubSubdir[1]:
        alldir = Normalsave_dir + '/' + x + '/' + y+ '/'
        print(alldir)
        DirectoryList.append(alldir)
        LabelList.append(Labels)
        
Subdir = next(os.walk(Apoptosissave_dir))

for x in Subdir[1]:
    currentdir = Apoptosissave_dir + '/' + x 
    CsvFile = sorted(glob(currentdir + '/' + '*.csv'))
    
    Labels = np.loadtxt(CsvFile[0], unpack = True)
    SubSubdir = next(os.walk(currentdir))
    for y in SubSubdir[1]:
        alldir = Apoptosissave_dir + '/' + x + '/' + y+ '/'
        DirectoryList.append(alldir)
        LabelList.append(Labels)
        
Subdir = next(os.walk(Divisionsave_dir))

for x in Subdir[1]:
    currentdir = Divisionsave_dir + '/' + x 
    CsvFile = sorted(glob(currentdir + '/' + '*.csv'))
    Labels = np.loadtxt(CsvFile[0], unpack = True)
    SubSubdir = next(os.walk(currentdir))
    for y in SubSubdir[1]:
        alldir = Divisionsave_dir + '/' + x + '/' + y+ '/'
        DirectoryList.append(alldir)
        LabelList.append(Labels)
        
Subdir = next(os.walk(Macrocheatesave_dir))

for x in Subdir[1]:
    currentdir = Macrocheatesave_dir + '/' + x 
    CsvFile = sorted(glob(currentdir + '/' + '*.csv'))
    Labels = np.loadtxt(CsvFile[0], unpack = True)
    SubSubdir = next(os.walk(currentdir))
    for y in SubSubdir[1]:
        alldir = Macrocheatesave_dir + '/' + x + '/' + y+ '/'
        DirectoryList.append(alldir)
        LabelList.append(Labels)
        
Subdir = next(os.walk(NonMaturesave_dir))

for x in Subdir[1]:
    currentdir = NonMaturesave_dir + '/' + x 
    CsvFile = sorted(glob(currentdir + '/' + '*.csv'))
    Labels = np.loadtxt(CsvFile[0], unpack = True)
    SubSubdir = next(os.walk(currentdir))
    for y in SubSubdir[1]:
        alldir = NonMaturesave_dir + '/' + x + '/' + y+ '/'
        DirectoryList.append(alldir)
        LabelList.append(Labels)
        
Subdir = next(os.walk(Maturesave_dir))

for x in Subdir[1]:
    currentdir = Maturesave_dir + '/' + x 
    CsvFile = sorted(glob(currentdir + '/' + '*.csv'))
    Labels = np.loadtxt(CsvFile[0], unpack = True)
    SubSubdir = next(os.walk(currentdir))
    for y in SubSubdir[1]:
        alldir = Maturesave_dir + '/' + x + '/' + y+ '/'
        DirectoryList.append(alldir)
        LabelList.append(Labels) 
        


/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/CenterONEAT/NormalEvents/NoShift/ResizeOriginal/
/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/CenterONEAT/NormalEvents/NoShift/Sigma1/


In [7]:
SaveName = 'CenterONEAT'
SaveNameVal = 'CenterONEATValidation'


MovieFrames = 9
npzfileGenerator.generate_training_data(DirectoryList, LabelList,SaveNpzDirectory, SaveName, SaveNameVal,0, MovieFrames, SizeX, SizeY)
        
SaveName = 'CenterONEATPrediction'
SaveNameVal = 'CenterONEATPredictionValidation'


MovieFrames = 5
npzfileGenerator.generate_training_data(DirectoryList, LabelList,SaveNpzDirectory, SaveName, SaveNameVal,0, MovieFrames, SizeX, SizeY)

SaveName = 'CenterONEATPrePrediction'
SaveNameVal = 'CenterONEATPrePredictionValidation'


MovieFrames = 3
npzfileGenerator.generate_training_data(DirectoryList, LabelList,SaveNpzDirectory, SaveName, SaveNameVal,0, MovieFrames, SizeX, SizeY)
    

/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/CenterONEAT/NormalEvents/NoShift/ResizeOriginal/


100%|██████████| 183/183 [00:00<00:00, 964.90it/s]


/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/CenterONEAT/NormalEvents/NoShift/Sigma1/


100%|██████████| 183/183 [00:00<00:00, 984.92it/s]


/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/CenterONEAT/ApoptosisEvents/NoShift/ResizeOriginal/


...

/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/CenterONEAT/NonMatureEvents/NoShift/ResizeOriginal/


100%|██████████| 400/400 [00:00<00:00, 1281.65it/s]


/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/CenterONEAT/NonMatureEvents/NoShift/Sigma1/


100%|██████████| 400/400 [00:00<00:00, 1050.15it/s]


(13045, 3, 54, 54, 1) (13045, 9, 1)
